#Chatgpt and langchain

In [ ]:
!pip install tiktoken
!pip install langchain==0.0.316
!pip install openai==0.28.1
!pip install python-dotenv
!pip install faiss-cpu
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 39.7 MB/s eta 0:00:00


In [ ]:
import openai
import os
import pandas as pd
from langchain import OpenAI
from dotenv import load_dotenv
from langchain.chains import RetrievalQA, LLMMathChain
from langchain.chat_models import ChatOpenAI
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from bs4 import BeautifulSoup
from typing import Union, Any, List, Optional
from langchain.document_loaders import (
    WebBaseLoader, PyMuPDFLoader, TextLoader, Docx2txtLoader,
    UnstructuredURLLoader, SeleniumURLLoader, PlaywrightURLLoader,
    AsyncHtmlLoader, AsyncChromiumLoader, UnstructuredMarkdownLoader,
    CSVLoader, JSONLoader, DataFrameLoader, UnstructuredImageLoader
    )
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownTextSplitter
from langchain.schema.document import Document
from langchain.evaluation.qa import QAGenerateChain
from langchain.chains.qa_generation.base import QAGenerationChain
from langchain.prompts import PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
import pickle
from langchain.chains.chat_vector_db.prompts import QA_PROMPT
from pathlib import Path
import pandas as pd
from langchain.callbacks import get_openai_callback
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import LLMChain
from langchain.chains.chat_vector_db.prompts import (
    CONDENSE_QUESTION_PROMPT, QA_PROMPT
)
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain, RetrievalQAWithSourcesChain
# from langchain.agents import create_csv_agent, create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
# import sharepy
import json

In [ ]:
_ = load_dotenv("credentials.env") # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

### Test chatbots

In [ ]:
# Define the folder path where you want to search for .md files
markdown_path = Path('sagemaker_documentation')

data = []
for file in markdown_path.glob('*.md'):
    loader = TextLoader(str(file))
    data += loader.load()

text_splitter = MarkdownTextSplitter(
    chunk_size=3000,
    chunk_overlap=200,
)
data = text_splitter.split_documents(data)

embeddings = OpenAIEmbeddings()
current_vectorstore = FAISS.from_documents(data, embeddings)

# # Save vectorstore
with open("sagemaker_documentation.pkl", "wb") as f:
    pickle.dump(current_vectorstore, f)

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import LLMChain
from langchain.chains.chat_vector_db.prompts import (
    CONDENSE_QUESTION_PROMPT, QA_PROMPT
)
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain, RetrievalQAWithSourcesChain


# ConversationalRetrievalChain
memory = ConversationBufferWindowMemory(
    memory_key="chat_history", return_messages=True, k=2
    )

question_gen_llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    verbose=False,
    max_retries=2,
    temperature=1
)
streaming_llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    # max_tokens=500,
    verbose=False,
    max_retries=2,
    temperature=1
)

question_generator = LLMChain(
    llm=question_gen_llm,
    prompt=CONDENSE_QUESTION_PROMPT,
    verbose=False
)

doc_chain = load_qa_chain(
    streaming_llm,
    chain_type="stuff",
    prompt=QA_PROMPT,
    verbose=False
)

qa = ConversationalRetrievalChain(
    retriever=current_vectorstore.as_retriever(search_kwargs={"k": 4}),
    combine_docs_chain=doc_chain,
    question_generator=question_generator,
    memory=memory,
    verbose=False,
)

In [ ]:
# qa.run("What is SageMaker?")
# qa.run("What are their Geospatial capabilities?")
qa.run("What is the cost")

'The cost of using Amazon SageMaker depends on various factors such as the instance type, storage, training time, and data transfer. It is best to refer to the AWS Pricing page for Amazon SageMaker or contact AWS support for specific pricing details.'

In [ ]:
# RetrievalQAWithSourcesChain
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    verbose=True,
    max_retries=1
)

qa = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    retriever=current_vectorstore.as_retriever(search_kwargs={"k": 5}),
    chain_type="stuff",
    return_source_documents=True,
)
qa("What are SageMaker Geospatial capabilities?", return_only_outputs=True)

In [ ]:
# qa("What is SageMaker?", return_only_outputs=True)
# qa("What are all AWS regions where AWS SageMaker is available?", return_only_outputs=True)
# qa("What do you know about AWS regions? is AWS SageMaker available on them?", return_only_outputs=True)
# qa("How to check if an endpoint is KMS encrypted?", return_only_outputs=True)
qa("What are SageMaker Geospatial capabilities?", return_only_outputs=True)

{'answer': 'The document does not provide information about SageMaker Geospatial capabilities.\n',
 'sources': 'N/A',
 'source_documents': [Document(page_content='SageMaker geospatial capabilities roles\n\nAs a managed service, Amazon SageMaker geospatial capabilities perform operations on your behalf on the AWS hardware that is managed by SageMaker. It can perform only operations that the user permits.\n\nA user can grant these permissions with an IAM role (referred to as an execution role).\n\nTo create and use a locally available execution role, you can use the following procedures.\n\nCreate an execution role\n\nTo work with SageMaker geospatial capabilities you need to setup a user role and an execution role. A user role is an AWS identity with permission policies that determine what the user can and can not do within AWS. An execution role is an IAM role that grants the service permission to access your AWS resources. An execution role consists of permissions and trust policy. Th

In [ ]:
# result = qa("What is SageMaker?", return_only_outputs=True)
# result = qa("What are all AWS regions where AWS SageMaker is available?", return_only_outputs=True)
# result = qa("How to check if an endpoint is KMS encrypted?", return_only_outputs=True)
result = qa("What are SageMaker Geospatial capabilities?", return_only_outputs=True)



> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.


## Test dataframe agent

In [ ]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI


df = pd.read_csv("unidadesnegocio.csv")

agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [ ]:
agent.run("¿Cuál es el precio del procedimiento USG abdomen inferior?")



> Entering new AgentExecutor chain...
El precio del procedimiento USG abdomen inferior es de 457.

> Finished chain.


'El precio del procedimiento USG abdomen inferior es de 457.'

In [ ]:
agent.run("¿Cuáles son los precios de un ultrasonido de mamas?")



> Entering new AgentExecutor chain...
Para obtener los precios de un ultrasonido de mamas en el dataframe `df`, podemos filtrar las filas que corresponden a esta categoría y luego seleccionar la columna de precios. Aquí está el código para hacerlo:

```python
precios_mamas = df[df['SUBCATEGORIA'] == 'USG MAMARIO']['PRECIO']
```

Esto creará una serie de pandas con los precios de los ultrasonidos de mamas. Puedes imprimir esta serie para ver los precios.

> Finished chain.


"Para obtener los precios de un ultrasonido de mamas en el dataframe `df`, podemos filtrar las filas que corresponden a esta categoría y luego seleccionar la columna de precios. Aquí está el código para hacerlo:\n\n```python\nprecios_mamas = df[df['SUBCATEGORIA'] == 'USG MAMARIO']['PRECIO']\n```\n\nEsto creará una serie de pandas con los precios de los ultrasonidos de mamas. Puedes imprimir esta serie para ver los precios."

In [ ]:
# agent.run("¿Cuáles son las unidades de negocio disponibles?")
# agent.run("¿Cuáles son las categorias disponibles dentro de la unidad de negocio RADIOLOGIA?")
agent.run("¿Cuál es el tiempo de entrega de un ultrasonido de mamas?")



> Entering new AgentExecutor chain...
El tiempo de entrega de un ultrasonido de mamas es "ENTREGA EN MOMENTO/ INFORME POR CORREO 1 HORA POST PROCEDIMIENTO".

> Finished chain.


'El tiempo de entrega de un ultrasonido de mamas es "ENTREGA EN MOMENTO/ INFORME POR CORREO 1 HORA POST PROCEDIMIENTO".'

In [ ]:
agent.run("¿Cuáles son los precios de una hematología completa y el precio de USG abdomen inferior?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `df[df['NOMBRE'] == 'HEMATOLOGIA COMPLETA']['PRECIO'].values[0]`


116.0
Invoking: `python_repl_ast` with `df[df['NOMBRE'] == 'USG ABDOMEN INFERIOR']['PRECIO'].values[0]`


457.0El precio de una hematología completa es de 116.0 y el precio de un USG abdomen inferior es de 457.0.

> Finished chain.


'El precio de una hematología completa es de 116.0 y el precio de un USG abdomen inferior es de 457.0.'

In [ ]:
agent.run("¿Cuáles es la suma de los precios de una hematología completa y de USG abdomen inferior y cuáles son sus precios individuales?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `df[df['NOMBRE'].isin(['HEMATOLOGIA COMPLETA', 'USG ABDOMEN INFERIOR'])]['PRECIO'].sum()`


573.0
Invoking: `python_repl_ast` with `df[df['NOMBRE'] == 'HEMATOLOGIA COMPLETA']['PRECIO'].values[0]`


116.0
Invoking: `python_repl_ast` with `df[df['NOMBRE'] == 'USG ABDOMEN INFERIOR']['PRECIO'].values[0]`


457.0La suma de los precios de una hematología completa y de USG abdomen inferior es de 573.0. 

El precio individual de la hematología completa es de 116.0 y el precio individual del USG abdomen inferior es de 457.0.

> Finished chain.


'La suma de los precios de una hematología completa y de USG abdomen inferior es de 573.0. \n\nEl precio individual de la hematología completa es de 116.0 y el precio individual del USG abdomen inferior es de 457.0.'

In [ ]:
agent.run("¿Puedes decirme toda la información posible de RESONANCIA MAGNETICA DE ABDOMEN SUPERIOR?")



> Entering new AgentExecutor chain...



Invoking: `python_repl_ast` with `{'query': "df[df['NOMBRE'] == 'RESONANCIA MAGNETICA DE ABDOMEN SUPERIOR']"}`


    UNIDAD DE NEGOCIO                   CATEGORIA SUBCATEGORIA  \
222        RADIOLOGIA  RESONANCIA MAGNETICA (RMN)      NINGUNA   

    REFERENCIA (SI/NO)                                    NOMBRE  \
222                 SI  RESONANCIA MAGNETICA DE ABDOMEN SUPERIOR   

    OTROS NOMBRES  PRECIO  PRECIO DESCUENTO VIVOLIFE  CODIGO SAM  \
222           NaN  2779.7                     2779.7      2680.0   

    OBSERVACIONES PREPARACION  \
222           NaN         NaN   

                                     TIEMPO DE ENTREGA  \
222  VARIA DE ACUERDO AL CENTRO DE DIAGNOSTICO AL C...   

                                       TIPO DE MUESTRA  \
222  SE REALIZA LECTURA DE FORMA PRESENCIAL CON INS...   

                                               CONTEXT  
222  informacion sobre precios de procedimientos, e...  La información disponible sobre "RESONANCIA MAGNETICA DE ABDOMEN 

'La información disponible sobre "RESONANCIA MAGNETICA DE ABDOMEN SUPERIOR" es la siguiente:\n\n- Unidad de Negocio: RADIOLOGIA\n- Categoría: RESONANCIA MAGNETICA (RMN)\n- Subcategoría: NINGUNA\n- Referencia (Si/No): SI\n- Nombre: RESONANCIA MAGNETICA DE ABDOMEN SUPERIOR\n- Otros Nombres: No se especifica\n- Precio: 2779.7\n- Precio Descuento Vivolife: 2779.7\n- Código SAM: 2680.0\n- Observaciones: No se especifica\n- Preparación: No se especifica\n- Tiempo de Entrega: VARIA DE ACUERDO AL CENTRO DE DIAGNOSTICO AL CUAL SE REFIERA\n- Tipo de Muestra: SE REALIZA LECTURA DE FORMA PRESENCIAL CON INSTRUMENTAL DE RADIOLOGIA\n- Contexto: Información sobre precios de procedimientos, exámenes'

In [ ]:
from typing import Dict
from langchain_core.tools import tool


@tool
def show_results_status(pacient_name: str) -> str:
    "Show the results of medical exams if client ask or assistant needs it"
    return f"""
    A continuación se muestran el estatus de los resultados para el paciente {pacient_name}

    Exámenes radiología:
    - Ultrasonografía pélvico: Completado
    - Ultrasonografía obstétrico: Completado

    Exámenes laboratorios:
    - Panel de parásitos: Completado
    - Helicobacter piloris: Pendiente
    - influenza de covid: Pendiente

    Informes médicos:
    - Informes: Pendientes

    De momento podemos están completos los examenes de radiología únicamente
    """

@tool
def validate_email(email: str) -> str:
    "When AI assistant needs to validate if the email is correct"
    return f"""
    Can you confirm if email is correct?:
      - {email}
    """

@tool
def send_results(pacient_name: str, email: str) -> str:
    "When the client confirms that all the email is correct, send the results"

    return f"""
    La información del paciente {pacient_name} ha sido enviada al correo {email}! Revisa tu correo en unos minutos
    y no te olvides de revisar el spam
    """

tools = [show_results_status, validate_email, send_results]

In [ ]:
PREFIX = """You are an Assistant that gives medical exams results.

When the client ask for results of medical exams, guide the client through next sequence of questions: \

First, ask the client the pacient name \
Second, show results status of the pacient name and ask if the client wants to recieve the exams results in his emal, if yes, ask the client the email \
Third, validate if the email is correct \

Finally, if the client confirms that the information is correct, send the results to the email.
"""

SUFFIX  = """Begin!
Previous conversation history:
{chat_history}
New input: {input}
{agent_scratchpad}"""

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
# from langchain.chat_models import ChatOpenAI

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
# prompt.pretty_print()-

In [ ]:
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

messages = [
            SystemMessagePromptTemplate.from_template(PREFIX),
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template("{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]

prompt.messages = messages

## Test agents

In [ ]:
from langchain.memory import ConversationBufferMemory


model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_tools_agent(model, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

memory = ConversationBufferMemory()
memory.buffer_as_messages

[]

In [ ]:
def invoke_agent(input: str):
    result = agent_executor.invoke(
        {
            "input": input,
            "chat_history": memory.buffer_as_messages
        }
    )

    result.pop('chat_history')
    memory.save_context({"input": result['input']}, {"output": result['output']})
    print(result['output'])

In [ ]:
invoke_agent('Hola buenas tardes')



> Entering new AgentExecutor chain...
¡Hola! Buenas tardes. ¿En qué puedo ayudarte hoy?

> Finished chain.
¡Hola! Buenas tardes. ¿En qué puedo ayudarte hoy?


In [ ]:
invoke_agent('quiero consultar el estado de mis examenes de laboratorio')



> Entering new AgentExecutor chain...
Claro, puedo ayudarte con eso. ¿Podrías proporcionarme el nombre del paciente?

> Finished chain.
Claro, puedo ayudarte con eso. ¿Podrías proporcionarme el nombre del paciente?


In [ ]:
invoke_agent('Luis Fernando Martinez López')



> Entering new AgentExecutor chain...

Invoking: `show_results_status` with `{'pacient_name': 'Luis Fernando Martinez López'}`



    A continuación se muestran el estatus de los resultados para el paciente Luis Fernando Martinez López

    Exámenes radiología:
    - Ultrasonografía pélvico: Completado
    - Ultrasonografía obstétrico: Completado

    Exámenes laboratorios:
    - Panel de parásitos: Completado
    - Helicobacter piloris: Pendiente
    - influenza de covid: Pendiente

    Informes médicos:
    - Informes: Pendientes

    De momento podemos están completos los examenes de radiología únicamente
    Los resultados de los exámenes de laboratorio para el paciente Luis Fernando Martinez López son los siguientes:

- Exámenes de radiología:
  - Ultrasonografía pélvica: Completado
  - Ultrasonografía obstétrica: Completado

- Exámenes de laboratorio:
  - Panel de parásitos: Completado
  - Helicobacter pylori: Pendiente
  - Influenza de COVID-19: Pendiente

- Informes médicos:


In [ ]:
invoke_agent('si por favor')



> Entering new AgentExecutor chain...
Perfecto. ¿Podrías proporcionarme tu dirección de correo electrónico para enviarte los resultados?

> Finished chain.
Perfecto. ¿Podrías proporcionarme tu dirección de correo electrónico para enviarte los resultados?


In [ ]:
invoke_agent('mi correo es pajarito@gmail.com')



> Entering new AgentExecutor chain...

Invoking: `validate_email` with `{'email': 'pajarito@gmail.com'}`



    Can you confirm if email is correct?:
      - pajarito@gmail.com
    ¿Podrías confirmar si el correo electrónico es correcto? El correo electrónico proporcionado es pajarito@gmail.com. ¿Es correcto?

> Finished chain.
¿Podrías confirmar si el correo electrónico es correcto? El correo electrónico proporcionado es pajarito@gmail.com. ¿Es correcto?


In [ ]:
invoke_agent('Si, es correcto')



> Entering new AgentExecutor chain...

Invoking: `send_results` with `{'pacient_name': 'Luis Fernando Martinez López', 'email': 'pajarito@gmail.com'}`



    La información del paciente Luis Fernando Martinez López ha sido enviada al correo pajarito@gmail.com! Revisa tu correo en unos minutos
    y no te olvides de revisar el spam
    ¡Perfecto! La información del paciente Luis Fernando Martinez López ha sido enviada al correo pajarito@gmail.com. Por favor, revisa tu correo en unos minutos y no olvides revisar la carpeta de spam por si acaso. Si tienes alguna otra pregunta, no dudes en hacerla.

> Finished chain.
¡Perfecto! La información del paciente Luis Fernando Martinez López ha sido enviada al correo pajarito@gmail.com. Por favor, revisa tu correo en unos minutos y no olvides revisar la carpeta de spam por si acaso. Si tienes alguna otra pregunta, no dudes en hacerla.
